## PL baseline calculation (raw data for forecast)

In [7]:
import pandas as pd 
import numpy as np
import calendar
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

In [8]:
folder_path = 'p:/FD-Supermarket/Бюджеты/Мот. бюджеты/auto_mot_budget/'
folder2 = 'Расчёт доли подстатьи/'
folder3 =  'Расчёт baseline pnl/'
folder4 =  'for info/'

basic_pnl = pd.read_excel(folder_path+folder3+'in_базовый pnl.xlsx', header=0)

In [9]:
#Transform the table
basic_pnl_2 = basic_pnl.melt(id_vars=['ЦФО', 'Название статьи узловая', 'Код статья узловая', 'Дата открытия', 'Дата закрытия',
                          'Дата начала реконструкции', 'Дата окончания реконструкции']).rename(
    columns={'variable':'Месяц', 'value':'Значение'})
#def month_to_date(column):
#    return datetime.strptime(column, "%d.%m.%Y")

In [11]:
#Calculate the total number of the days in a month
def days_count(table):
    date = table['Месяц']
    return calendar.monthrange(date.year, date.month)[1]
basic_pnl_2['Кол-во дней в месяце'] = basic_pnl_2.apply(days_count, axis=1)

In [12]:
#Add last day of the month columns
def end_of_the_month(table):
    start_of_the_month, total_days = table['Месяц'], table['Кол-во дней в месяце']
    return start_of_the_month.replace(day=total_days)
basic_pnl_2['Конец месяца'] = basic_pnl_2.apply(end_of_the_month, axis=1)

In [13]:
#Calculate the number of working days:
def active_days(table):
    start_of_the_month, end_of_the_month = table['Месяц'], table['Конец месяца']
    total_days = table['Кол-во дней в месяце']
    opening_date, closing_date = table['Дата открытия'], table['Дата закрытия']
    start_of_the_rcr = table['Дата начала реконструкции'] 
    finish_of_the_rcr = table['Дата окончания реконструкции']
    if start_of_the_rcr >= start_of_the_month and start_of_the_rcr <= end_of_the_month:
        active_days = (start_of_the_rcr - start_of_the_month).days
    elif finish_of_the_rcr >= start_of_the_month and finish_of_the_rcr <= end_of_the_month:
        active_days = (end_of_the_month - finish_of_the_rcr).days + 1
    elif closing_date >= start_of_the_month and closing_date <= end_of_the_month:
        active_days = (closing_date - start_of_the_month).days
    elif opening_date >= start_of_the_month and opening_date <= end_of_the_month:
        active_days = (end_of_the_month - opening_date).days + 1
    elif start_of_the_rcr < start_of_the_month and finish_of_the_rcr > end_of_the_month:
        active_days = 0
    elif start_of_the_month > closing_date:
        active_days = 0
    elif end_of_the_month < opening_date:
        active_days = 0
    else: 
        active_days = total_days
    return active_days
basic_pnl_2['Рабочие дни'] = basic_pnl_2.apply(active_days, axis=1)

In [14]:
#Add "full month" attribute
basic_pnl_2['Полный месяц'] = basic_pnl_2['Кол-во дней в месяце'] == basic_pnl_2['Рабочие дни']

In [15]:
#Create a table including only the full months
basic_pnl_full_month = basic_pnl_2[basic_pnl_2['Полный месяц']==True]
#Calculate the values for full months
basic_pnl_full_month2 = basic_pnl_full_month.groupby(
    ['ЦФО', 'Код статья узловая'], as_index=False).agg({'Значение':'mean'})
basic_pnl_full_month2 = basic_pnl_full_month2.rename(columns={'Значение':'Среднемесячное значение'})

In [16]:
#Add monthly average values to the budget table
basic_pnl_4 = basic_pnl_2.merge(basic_pnl_full_month2, on=['ЦФО', 'Код статья узловая'], how='left')
#Fill the column with avg  Заполняем столбец значений средними там, где неполный месяц
def avg_month(table):
    value = table['Значение']
    avg_value = table['Среднемесячное значение']
    if table['Полный месяц'] == False:
        return avg_value
    else: 
        return value
basic_pnl_4['Итоговое значение'] = basic_pnl_4.apply(avg_month, axis=1)

In [17]:
basic_pnl_5 = basic_pnl_4[['ЦФО', 'Код статья узловая', 'Месяц', 'Итоговое значение']]
basic_pnl_5.to_csv(
    folder_path+folder3+'out_baseline_pnl.csv', index=False, encoding='cp1251',sep = ';')